In [1]:
from skimage import transform,exposure
from sklearn import model_selection, preprocessing, metrics, feature_selection
import os
import time
import numpy as np
import pandas as pd
import torch
from sklearn.preprocessing import MinMaxScaler
from torch.utils import data as torch_data
from torch.nn import functional as torch_functional

from MRI_package.myDataset import GetLoader
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from MRI_package.utils2 import mkdir,load_npy_data,calculate,_init_fn,set_seed

In [19]:
dir="/home/linjiatai/Research/Data/model_data/xlsx"
MRI_dir="/home/linjiatai/Research/Data/MRI"

In [57]:
meta=pd.read_excel(os.path.join(MRI_dir,"protein","model_meta_fal.xlsx"))
meta_train=pd.read_excel(os.path.join(dir,"meta_train.xlsx"))
meta_test=pd.read_excel(os.path.join(dir,"meta_test.xlsx"))
MRI_lasso_train=pd.read_excel(os.path.join(dir,"MRI_lasso_train.xlsx"))
MRI_lasso_test=pd.read_excel(os.path.join(dir,"MRI_lasso_test.xlsx"))
MRI_ttest_train=pd.read_excel(os.path.join(dir,"MRI_ttest_train.xlsx"))
MRI_ttest_test=pd.read_excel(os.path.join(dir,"MRI_ttest_test.xlsx"))
protein_train=pd.read_excel(os.path.join(dir,"protein_train.xlsx"))
protein_test=pd.read_excel(os.path.join(dir,"protein_test.xlsx"))

MRI_train=meta_train["MRI"]
MRI_test=meta_test["MRI"]
y_train=meta_train["pCR"]
y_test=meta_test["pCR"]
meta_train=meta_train.iloc[:,2:]
meta_test=meta_train.iloc[:,2:]
protein_train=protein_train.iloc[:,1:]
protein_test=protein_test.iloc[:,1:]
MRI_lasso_train=MRI_lasso_train.iloc[:,1:]
MRI_lasso_test=MRI_lasso_test.iloc[:,1:]
MRI_ttest_train=MRI_ttest_train.iloc[:,1:]
MRI_ttest_test=MRI_ttest_test.iloc[:,1:]

In [58]:
meta=meta.set_index("MRI")
meta=meta.iloc[:,1:]

In [59]:
scaler = MinMaxScaler()
protein_train=scaler.fit_transform(protein_train)
protein_test=scaler.fit_transform(protein_test)

In [60]:
meta1=pd.get_dummies(meta)
scaler = MinMaxScaler()
# 对DataFrame中的每一列进行归一化
meta2= scaler.fit_transform(meta1)

In [61]:
meta2=pd.DataFrame(meta2)
meta2.columns=meta1.columns
meta2.index=meta1.index

In [62]:
meta_train=meta2.loc[MRI_train]
meta_test=meta2.loc[MRI_test]

In [63]:
#np读取数据
datanp_train=load_npy_data(MRI_train,MRI_dir,"1")
datanp_val=load_npy_data(MRI_test,MRI_dir,"1")

loading....
11698751
before resize shape (55, 64, 66)
11496266
before resize shape (47, 34, 34)
11337169
before resize shape (41, 85, 51)
11678836
before resize shape (32, 36, 28)
11543998
before resize shape (68, 52, 68)
11514471
before resize shape (28, 21, 24)
11499914
before resize shape (70, 100, 83)
11456908
before resize shape (96, 117, 86)
11731455
before resize shape (69, 72, 80)
10976546
before resize shape (40, 68, 47)
11404064
before resize shape (36, 62, 30)
11684525
before resize shape (21, 16, 20)
11576884
before resize shape (30, 52, 21)
11763455
before resize shape (65, 96, 52)
11288231
before resize shape (30, 32, 80)
11749792
before resize shape (40, 51, 41)
11277955
before resize shape (23, 26, 45)
11453290
before resize shape (68, 45, 127)
10687794
before resize shape (55, 38, 50)
10752825
before resize shape (54, 53, 39)
11692549
before resize shape (216, 114, 76)
10607177
before resize shape (47, 47, 49)
11414198
before resize shape (27, 24, 21)
11391454
before r

KeyboardInterrupt: 

In [64]:
clin_train1=meta_train.values
clin_test1=meta_test.values
MRI_lasso_train1=MRI_lasso_train.values
MRI_lasso_test1=MRI_lasso_test.values
MRI_ttest_train1=MRI_ttest_train.values
MRI_ttest_test1=MRI_ttest_test.values
y_train1=y_train.values
y_test1=y_test.values

In [65]:
clin_train1=torch.tensor(clin_train1)
clin_test1=torch.tensor(clin_test1)
protein_train1=torch.tensor(protein_train)
protein_test1=torch.tensor(protein_test)
MRI_lasso_train1=torch.tensor(MRI_lasso_train1)
MRI_lasso_test1=torch.tensor(MRI_lasso_test1)
MRI_ttest_train1=torch.tensor(MRI_ttest_train1)
MRI_ttest_test1=torch.tensor(MRI_ttest_test1)
# MRI_train1=torch.tensor(datanp_train)
# MRI_test1=torch.tensor(datanp_val)
y_train1=torch.tensor(y_train1)
y_test1=torch.tensor(y_test1)

In [66]:
torch.save(clin_train1,"clin_train.pt")
torch.save(clin_test1,"clin_test.pt")
torch.save(protein_train1,"protein_train.pt")
torch.save(protein_test1,"protein_test.pt")
torch.save(MRI_lasso_train1,"MRI_lasso_train.pt")
torch.save(MRI_lasso_test1,"MRI_lasso_test.pt")
# torch.save(MRI_ttest_train1,"MRI_ttest_train.pt")
# torch.save(MRI_ttest_test1,"MRI_ttest_test.pt")
torch.save(MRI_train1,"MRI_train.pt")
torch.save(MRI_test1,"MRI_test.pt")
torch.save(y_train1,"y_train.pt")
torch.save(y_test1,"y_test.pt")